In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np
from datetime import timedelta
from ta import add_all_ta_features
from ta.utils import dropna
from data_process_helpers import *

# get data in desirved format
dow_files_addr = "./Data/Dow_30_1_min/"
etf_files_addr = "./Data/50_ETFs_1min/"

dow_files_names = [f for f in listdir(dow_files_addr)]
etf_files_names = [f for f in listdir(etf_files_addr)]

dow_names = [f.split(".")[0] for f in dow_files_names]
etf_names = [f.split(".")[0] for f in etf_files_names]

dow_dfs = []
etf_dfs = []
headers = ["date", "time", "open_price", "close_price",
           "highest_price", "lowest_price", "volumn"]
dtypes = {"date": "str",
          "time": "str",
          "open_price": "float",
          "highest_price": "float",
          "lowest_price": "float",
          "close_price": "float",
          "volumn": "int"}

print("Start read dow file...")
dow_dfs = read_files(dow_files_names, dow_files_addr, headers, dtypes)
print("Finish read dow file.")
print("---------------------")
print("Start read etf file...")
etf_dfs = read_files(etf_files_names, etf_files_addr, headers, dtypes)
print("Finish read etf file.")


# get start and end for both dow and etf that cove evey df
dow_time_slice = [min(dow_dfs[0]["datetime"]), max(dow_dfs[0]["datetime"])]
etf_time_slice = [min(etf_dfs[0]["datetime"]), max(etf_dfs[0]["datetime"])]

print("Start check dow date...")
dow_time_slice = find_common_time_slice(
    dow_dfs, dow_time_slice, dow_files_names)
print("Finish check dow date.")
print("---------------------")
print("Start check etf date...")
etf_time_slice = find_common_time_slice(
    etf_dfs, etf_time_slice, etf_files_names)
print("Finish check etf date.")


# limit all of the df to the same time frames
print("Start fixing time range...")
for i in range(len(dow_dfs)):
    dow_dfs[i] = dow_dfs[i].loc[(dow_dfs[i]["datetime"] >= dow_time_slice[0]) & (
        dow_dfs[i]["datetime"] <= dow_time_slice[1])]
for i in range(len(etf_dfs)):
    etf_dfs[i] = etf_dfs[i].loc[(etf_dfs[i]["datetime"] >= etf_time_slice[0]) & (
        etf_dfs[i]["datetime"] <= etf_time_slice[1])]
print("Finish fixing time range.")




Start read dow file...
Finish read dow file.
---------------------
Start read etf file...
Finish read etf file.
Start check dow date...
start GM.txt
2010-11-18 09:36:00
end GM.txt
2020-06-05 19:44:00
Finish check dow date.
---------------------
Start check etf date...
start EEM.txt
2003-04-14 09:32:00
end EEM.txt
2020-06-05 19:41:00
end EFA.txt
2020-06-05 17:38:00
start ERX.txt
2008-11-19 09:30:00
start UPRO.txt
2009-06-25 09:40:00
Finish check etf date.
Start fixing time range...
Finish fixing time range.


In [2]:
# Modify all time difference to 5 min
print("Start modify dow time difference...")
dow_5min_dfs = modify_time_inteval(dow_dfs, dow_time_slice, 5)
print("Finish modify dow time difference.")

print("Start modify etf time difference...")
etf_5min_dfs = modify_time_inteval(etf_dfs, etf_time_slice, 5)
print("Finish modify etf time difference.")

Start modify dow time difference...
df:  0


KeyboardInterrupt: 